In [ ]:
%pip install -qU langchain langchain-openai

%pip install langchain-google-genai

%pip install --upgrade --quiet llamaapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.3 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.prompts import ChatPromptTemplate

os.environ['GOOGLE_API_KEY'] = "your-api-key"

# Initialize an LLM
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,)

# see the list of models here : https://ai.google.dev/gemini-api/docs/models/gemini


In [ ]:
# To use llama (no tool binding available for llama)
# %pip install langchain-experimental

# from llamaapi import LlamaAPI
# from langchain_experimental.llms import ChatLlamaAPI

# # Replace 'Your_API_Token' with your actual API token
# llama = LlamaAPI("api-key")

# llm = ChatLlamaAPI(client=llama)

In [ ]:
import datetime
from typing import Literal, Optional, Tuple

from langchain_core.pydantic_v1 import BaseModel, Field


class SubQuery(BaseModel):
    """Subtask of the original query which can't be broken further, and that which can be solved by an expert finetuned LLM."""

    sub_query: str = Field(
        ...,
        description="The minimal and specific subtask that can be solved by an expert finetuned LLM.",
    )

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage


system = """You are a helpful assistant that prepares queries that will be sent to a search component.
Sometimes, these queries are very complex.
Your job is to simplify complex queries into multiple queries that can be answered
in isolation to eachother.
"""
# # This is a prompt template used to format each individual example.
# example_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("human", "{question}"),
#         ("ai", "{output}"),
#     ]
# )

# examples = [
#     {"question": "Who lived longer, Muhammad Ali or Alan Turing?", "output": """Subtask1: How old was Muhammad Ali when he died? Subtask2: How old was Alan Turing when he died?"""},
#     {"question": "When was the founder of craigslist born?", "output": """
#     Subtask1: Who was the founder of craigslist?
#     Intermediate answer: Craigslist was founded by Craig Newmark.
#     Subtask2: When was Craig Newmark born?
#     Intermediate answer: Craig Newmark was born on December 6, 1952"""},
#     {"question": "Who is the current US President's wife?",
#      "output": "Subquery1 = Who is the current US president? Intermediate Answer : Donald Trump. Subquery 2 = Who is the wife of Donald Trump? Answer:Melania Trump. You will return Subquery 1 and Subquery 2 as the output."},

# ]

# few_shot_prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples,
# )


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        # few_shot_prompt,
        ("human", "{question}"),
    ]
)

llm_with_tools = llm.bind_tools([SubQuery])
parser = PydanticToolsParser(tools=[SubQuery])
query_analyzer = prompt | llm_with_tools | parser

In [ ]:
question = 'What is the capital of France? Is it the birthplace of Napoleon? Was he ever the emperor?'
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query='What is the capital of France?'),
 SubQuery(sub_query='Is Paris the birthplace of Napoleon?'),
 SubQuery(sub_query='Was Napoleon ever the emperor?')]

In [ ]:
question = "Who lived longer, Muhammad Ali or Alan Turing?"
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query='How long did Muhammad Ali live?'),
 SubQuery(sub_query='How long did Alan Turing live?')]

In [ ]:
question = "When was the founder of craigslist born?"
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query='Who is the founder of craigslist?'),
 SubQuery(sub_query='When was Craig Newmark born?')]

In [ ]:
question = "What is the formula for Einsten's most famous equation and is it related to gravity?"
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query="What is the formula for Einstein\\'s most famous equation?"),
 SubQuery(sub_query="Is Einstein\\'s most famous equation related to gravity?")]

In [ ]:
question = "Return the first letter of each word in this sentence 'hi how are you?' "
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query='What is the first letter of the word "hi"?'),
 SubQuery(sub_query='What is the first letter of the word "how"?'),
 SubQuery(sub_query='What is the first letter of the word "are"?'),
 SubQuery(sub_query='What is the first letter of the word "you"?')]

In [ ]:
question = 'This math problem has got me stumped: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? Can you show me the way?'
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query='How many clips did Natalia sell to her friends in April?'),
 SubQuery(sub_query='How many clips did Natalia sell to her friends in May?'),
 SubQuery(sub_query='If Natalia sold some number of clips in April and another number of clips in May, how many clips did she sell in total?')]

In [ ]:
question = 'Can you break down the solution for me for this math problem: Mr. Sam shared a certain amount of money between his two sons, Ken and Tony. If Ken got $1750, and Tony got twice as much as Ken, how much was the money shared?'
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query='Calculate twice the amount of $1750.'),
 SubQuery(sub_query='Calculate the sum of $1750 and $3500.')]